# Attractiveness
The efficiency of the route – does this route directly transport to places people want to go?

Stops are considered attractive if they allow users to travel to population centres. This means that while a stop may be perhaps mid-nowhere, it will still get a high score if a line served transports to a population centre.

In [1]:
%load_ext autoreload
%autoreload 2

from scripts.preprocessing import stop_data
from Transit_Quality_Study.transit_quality_study import config

Initializing GTFS feed...
Processing GTFS files...
Parquet files found. Loading parquet files.
Reading census files...
Success.
Census files merged.


# Map highest stop density to entire route

In [2]:
# Let's get the info that matters
route_max_density = stop_data[['route_id', 'Density']]
# Split stops with multiple lines to multiple rows
route_max_density = route_max_density.explode('route_id')
# Group the lines, keep the highest stop density
route_max_density = route_max_density.groupby('route_id').max('Density')
route_max_density = route_max_density.rename(columns = {'Density':'Max_Density'})

route_max_density.head()

,Max_Density
route_id,
1,18209.0
10,20000.0
100,32455.5
101,31205.2
102,20782.8


# Inherit route density in stop

In [3]:
stop_max_density = stop_data[['route_id']]
stop_max_density['stop_code'] = stop_max_density.index
stop_max_density = (stop_max_density
                    .explode('route_id')
                    .merge(route_max_density, on='route_id', how='left')
# If a stop has multiple routes, it inherits the maximum density
                    .groupby('stop_code').max('Max_Density'))
stop_max_density.head()

,Max_Density
stop_code,
10118,18209.0
10120,18209.0
10122,18209.0
10124,18209.0
10126,18209.0


In [4]:
stop_max_density.to_parquet(config.processed_data_dir / 'stop_max_density.parquet')